In [31]:
## import tensorflow as tf
import os
import gzip
import os
import sys
import urllib
import matplotlib.image as mpimg
import numpy as np
from PIL import Image

import code

import tensorflow.python.platform

import numpy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D


NUM_CHANNELS = 3  # RGB images
PIXEL_DEPTH = 255
NUM_LABELS = 2
TRAINING_SIZE = 20
VALIDATION_SIZE = 5  # Size of the validation set.
SEED = 66478  # Set to None for random seed.
BATCH_SIZE = 16  # 64
NUM_EPOCHS = 100
RESTORE_MODEL = True  # If True, restore existing model instead of training a new one
RECORDING_STEP = 0
IMG_PATCH_SIZE = 16
root_dir = "/Users/abiola/Documents/EPFL/ML_Project2_Road_Segmentation/Ressources/training/"

# Extract patches from a given image
def img_crop(im, w, h):
    list_patches = []
    imgwidth = im.shape[0]
    imgheight = im.shape[1]
    is_2d = len(im.shape) < 3
    for i in range(0, imgheight, h):
        for j in range(0, imgwidth, w):
            if is_2d:
                im_patch = im[j:j+w, i:i+h]
            else:
                im_patch = im[j:j+w, i:i+h, :]
            list_patches.append(im_patch)
    return list_patches

def load_image(infilename):
    data = mpimg.imread(infilename)
    return data


def extract_data(filename, num_images):
    """Extract the images into a 4D tensor [image index, y, x, channels].
    Values are rescaled from [0, 255] down to [-0.5, 0.5].
    """
    imgs = []
    image_dir = root_dir + "out/images/"
    files = os.listdir(image_dir)
    n = num_images
    print("Loading " + str(n) + " images")
    imgs = [load_image(image_dir + files[i])[:,:,:3] for i in range(n)]
    imgs = np.asarray(imgs)
    num_images = len(imgs)
    IMG_WIDTH = imgs[0].shape[0]
    IMG_HEIGHT = imgs[0].shape[1]
    N_PATCHES_PER_IMAGE = (IMG_WIDTH/IMG_PATCH_SIZE)*(IMG_HEIGHT/IMG_PATCH_SIZE)

    print("NB patches / imgae: ", N_PATCHES_PER_IMAGE)

    img_patches = [img_crop(imgs[i], IMG_PATCH_SIZE, IMG_PATCH_SIZE) for i in range(num_images)]
    data = [img_patches[i][j] for i in range(len(img_patches)) for j in range(len(img_patches[i]))]

    return numpy.asarray(data)



# Assign a label to a patch v
def value_to_class(v):
    foreground_threshold = 0.25  # percentage of pixels > 1 required to assign a foreground label to a patch
    df = numpy.sum(v)
    if df > foreground_threshold:  # road
        return [0, 1]
    else:  # bgrd
        return [1, 0]


    # Extract label images
def extract_labels(filename, num_images):
    """Extract the labels into a 1-hot matrix [image index, label index]."""
    gt_imgs = []
    gt_dir = root_dir + "out/groundtruth/"
    files = os.listdir(gt_dir)

    n = num_images
    print("Loading " + str(n) + " images")
    gt_imgs = [load_image(gt_dir + files[i])[:,:,0] for i in range(n)]
    gt_imgs = np.asarray(gt_imgs)
    
    print(gt_imgs[0])

    num_images = len(gt_imgs)
    gt_patches = [img_crop(gt_imgs[i], IMG_PATCH_SIZE, IMG_PATCH_SIZE) for i in range(num_images)]
    data = numpy.asarray([gt_patches[i][j] for i in range(len(gt_patches)) for j in range(len(gt_patches[i]))])
    labels = numpy.asarray([value_to_class(numpy.mean(data[i])) for i in range(len(data))])

    # Convert to dense 1-hot representation.
    return labels.astype(numpy.float32)


def error_rate(predictions, labels):
    """Return the error rate based on dense predictions and 1-hot labels."""
    return 100.0 - (
        100.0 *
        numpy.sum(numpy.argmax(predictions, 1) == numpy.argmax(labels, 1)) /
        predictions.shape[0])

# Write predictions from neural network to a file
def write_predictions_to_file(predictions, labels, filename):
    max_labels = numpy.argmax(labels, 1)
    max_predictions = numpy.argmax(predictions, 1)
    file = open(filename, "w")
    n = predictions.shape[0]
    for i in range(0, n):
        file.write(max_labels(i) + ' ' + max_predictions(i))
    file.close()


    
# Print predictions from neural network
def print_predictions(predictions, labels):
    max_labels = numpy.argmax(labels, 1)
    max_predictions = numpy.argmax(predictions, 1)
    print(str(max_labels) + ' ' + str(max_predictions))


# Convert array of labels to an image
def label_to_img(imgwidth, imgheight, w, h, labels):
    array_labels = numpy.zeros([imgwidth, imgheight])
    idx = 0
    for i in range(0, imgheight, h):
        for j in range(0, imgwidth, w):
            if labels[idx][0] > 0.5:  # bgrd
                l = 0
            else:
                l = 1
            array_labels[j:j+w, i:i+h] = l
            idx = idx + 1
    return array_labels


def img_float_to_uint8(img):
    rimg = img - numpy.min(img)
    rimg = (rimg / numpy.max(rimg) * PIXEL_DEPTH).round().astype(numpy.uint8)
    return rimg


def concatenate_images(img, gt_img):
    n_channels = len(gt_img.shape)
    w = gt_img.shape[0]
    h = gt_img.shape[1]
    if n_channels == 3:
        cimg = numpy.concatenate((img, gt_img), axis=1)
    else:
        gt_img_3c = numpy.zeros((w, h, 3), dtype=numpy.uint8)
        gt_img8 = img_float_to_uint8(gt_img)          
        gt_img_3c[:, :, 0] = gt_img8
        gt_img_3c[:, :, 1] = gt_img8
        gt_img_3c[:, :, 2] = gt_img8
        img8 = img_float_to_uint8(img)
        cimg = numpy.concatenate((img8, gt_img_3c), axis=1)
    return cimg


def make_img_overlay(img, predicted_img):
    w = img.shape[0]
    h = img.shape[1]
    color_mask = numpy.zeros((w, h, 3), dtype=numpy.uint8)
    color_mask[:, :, 0] = predicted_img*PIXEL_DEPTH

    img8 = img_float_to_uint8(img)
    background = Image.fromarray(img8, 'RGB').convert("RGBA")
    overlay = Image.fromarray(color_mask, 'RGB').convert("RGBA")
    new_img = Image.blend(background, overlay, 0.2)
    return new_img


In [33]:
# Extract it into numpy arrays.

data_dir = 'training/'
train_data_filename = data_dir + 'images/'
train_labels_filename = data_dir + 'groundtruth/' 

X = extract_data(train_data_filename, TRAINING_SIZE)
y = extract_labels(train_labels_filename, TRAINING_SIZE)

print(y.shape)
print(X.shape)

Loading 20 images
NB patches / imgae:  625.0
Loading 20 images
[[0.07843138 0.07843138 0.07843138 ... 0.07843138 0.07843138 0.07843138]
 [0.07843138 0.07843138 0.07843138 ... 0.07843138 0.07843138 0.07843138]
 [0.07843138 0.07843138 0.07843138 ... 0.07843138 0.07843138 0.07843138]
 ...
 [0.07843138 0.07843138 0.07843138 ... 0.07843138 0.07843138 0.07843138]
 [0.07843138 0.07843138 0.07843138 ... 0.07843138 0.07843138 0.07843138]
 [0.07843138 0.07843138 0.07843138 ... 0.07843138 0.07843138 0.07843138]]
(12500, 2)
(12500, 16, 16, 3)


In [27]:
# Blance training data 
c0 = 0  # bgrd
c1 = 0  # road

for i in range(len(y)):
    if y[i][0] == 1:
        c0 = c0 + 1
    else:
        c1 = c1 + 1
print('Number of data points per class: c0 = ' + str(c0) + ' c1 = ' + str(c1))
print('Balancing training data...')

min_c = min(c0, c1)
idx0 = [i for i, j in enumerate(y) if j[0] == 1]
idx1 = [i for i, j in enumerate(y) if j[1] == 1]
new_indices = idx0[0:min_c] + idx1[0:min_c]


X = X[new_indices, :, :, :]
y = y[new_indices]

Number of data points per class: c0 = 3046 c1 = 3046
Balancing training data...


In [28]:
model = Sequential()

model.add(Conv2D(64,(3,3), input_shape = X.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(64,(3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Flatten())
model.add(Dense(64))

model.add(Dense(1))
model.add(Activation("sigmoid"))

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metric=["accuracy"])


In [29]:
model.fit(X,y, batch_size = BATCH_SIZE, validation_split = 0.1)

ValueError: Error when checking target: expected activation_17 to have shape (1,) but got array with shape (2,)